# Vector calculus in Cartesian and spherical coordinates

*This worksheet illustrates some features of [SageManifolds](http://sagemanifolds.obspm.fr/) (v0.9.1) regarding vector calculus in the Euclidean 3-space by means of Cartesian and spherical coordinates.*

Click [here](https://raw.githubusercontent.com/sagemanifolds/SageManifolds/master/Worksheets/v0.9.1/SM_Cartesian_spherical-3D.ipynb) to download the worksheet file (ipynb format). To run it, you must start SageMath with the Jupyter notebook, via the command `sage -n jupyter`

## Euclidean 3-space and Cartesian coordinates

First we set up the notebook to display mathematical objects using LaTeX formatting:

In [1]:
%display latex

We then introduce the Euclidean space as a 3-dimensional differentiable manifold:

In [2]:
M = Manifold(3, 'M', start_index=1)
print(M)

3-dimensional differentiable manifold M


We then introduce the Cartesian coordinates $(x,y,z)$ as the chart `cart` on $M$:

In [3]:
cart.<x,y,z> = M.chart()
print(cart)
cart

Chart (M, (x, y, z))


Chart (M, (x, y, z))

## Spherical coordinates

We introduce spherical coordinates $(r,\theta,\phi)$ as the chart `spher` on $M$:

In [4]:
spher.<r,th,ph> = M.chart(r'r:(0,+oo) th:(0,pi):\theta ph:(0,2*pi):\phi')
print(spher)
spher

Chart (M, (r, th, ph))


Chart (M, (r, th, ph))

Spherical coordinates do not form a regular coordinate system of the Euclidean space. So declaring that they span $M$ means that, strictly speaking, the manifold $M$ is not the whole Euclidean space, but the Euclidean space minus some half plane (the azimuthal origin). However, in this worksheet, this difference will not matter. 

The change of coordinates $(r,\theta,\phi) \rightarrow (x,y,z)$ is introduced as a transition map from chart `spher` to chart `cart`:

In [5]:
spher_to_cart = spher.transition_map(cart, [r*sin(th)*cos(ph), r*sin(th)*sin(ph), r*cos(th)])
spher_to_cart.display()

x = r*cos(ph)*sin(th)
y = r*sin(ph)*sin(th)
z = r*cos(th)

The inverse is also set:

In [6]:
spher_to_cart.set_inverse(sqrt(x^2+y^2+z^2), atan2(sqrt(x^2+y^2),z), atan2(y, x),
                          verbose=True)

Check of the inverse coordinate transformation:
  r == r
  th == arctan2(r*sin(th), r*cos(th))
  ph == arctan2(r*sin(ph)*sin(th), r*cos(ph)*sin(th))
  x == x
  y == y
  z == z


The check that the provided formulas do correspond to the inverse change of coordinates is passed, modulo some lack of simplification in some trigonometrical formulas involving the function `arctan2`. 

In [7]:
cart_to_spher = spher_to_cart.inverse()
cart_to_spher.display()

r = sqrt(x^2 + y^2 + z^2)
th = arctan2(sqrt(x^2 + y^2), z)
ph = arctan2(y, x)

The natural vector frame of spherical coordinates is

In [8]:
spher.frame()

Coordinate frame (M, (d/dr,d/dth,d/dph))

We shall expand vector and tensor fields on the orthonormal frame $(e_1, e_2, e_3)$ associated with spherical coordinates, which is related to the natural frame $(\partial/\partial r, \partial/\partial\theta, \partial/\partial\phi)$ displayed above by means of the following field of automorphisms: 

In [9]:
to_orthonormal = M.automorphism_field()
to_orthonormal[spher.frame(),1,1,spher] = 1
to_orthonormal[spher.frame(),2,2,spher] = 1/r
to_orthonormal[spher.frame(),3,3,spher] = 1/(r*sin(th))
to_orthonormal.display(spher.frame(), spher)

d/dr*dr + 1/r d/dth*dth + 1/(r*sin(th)) d/dph*dph

In other words, the change-of-basis matrix is 

In [10]:
to_orthonormal[spher.frame(),:,spher] 

[            1             0             0]
[            0           1/r             0]
[            0             0 1/(r*sin(th))]

We construct the orthonormal frame from the natural frame of spherical coordinates by this change of basis:

In [11]:
es = spher.frame().new_frame(to_orthonormal, 'e')
print(es)
es

Vector frame (M, (e_1,e_2,e_3))


Vector frame (M, (e_1,e_2,e_3))

In [12]:
es[1].display(spher.frame(), spher)

e_1 = d/dr

In [13]:
es[2].display(spher.frame(), spher)

e_2 = 1/r d/dth

In [14]:
es[3].display(spher.frame(), spher)

e_3 = 1/(r*sin(th)) d/dph

If we do not specify the frame and coordinates for the display, we get it in terms of the default ones (i.e. Cartesian frame and Cartesian coordinates):

In [15]:
es[1].display()

e_1 = x/sqrt(x^2 + y^2 + z^2) d/dx + y/sqrt(x^2 + y^2 + z^2) d/dy + z/sqrt(x^2 + y^2 + z^2) d/dz

In [16]:
es[2].display()

e_2 = x*z/(sqrt(x^2 + y^2 + z^2)*sqrt(x^2 + y^2)) d/dx + y*z/(sqrt(x^2 + y^2 + z^2)*sqrt(x^2 + y^2)) d/dy - sqrt(x^2 + y^2)/sqrt(x^2 + y^2 + z^2) d/dz

In [17]:
es[3].display()

e_3 = -y/sqrt(x^2 + y^2) d/dx + x/sqrt(x^2 + y^2) d/dy

By construction, the change of frame $(\partial/\partial r, \partial/\partial \theta, \partial/\partial \phi) \rightarrow (e_1,e_2,e_3)$ is known. We form the change of frame  $(\partial/\partial x, \partial/\partial y, \partial/\partial z)\rightarrow (e_1,e_2,e_3)$ by composition of $(\partial/\partial x, \partial/\partial y, \partial/\partial z)\rightarrow (\partial/\partial r, \partial/\partial \theta, \partial/\partial \phi)$ with 
$(\partial/\partial r, \partial/\partial \theta, \partial/\partial \phi) \rightarrow (e_1,e_2,e_3)$:

In [18]:
M.set_change_of_frame(cart.frame(), es, 
                      M.change_of_frame(spher.frame(), es) * M.change_of_frame(cart.frame(), spher.frame()),
                      compute_inverse=False)
M.change_of_frame(cart.frame(), es)[:, spher]

[cos(ph)*sin(th) cos(ph)*cos(th)        -sin(ph)]
[sin(ph)*sin(th) cos(th)*sin(ph)         cos(ph)]
[        cos(th)        -sin(th)               0]

Similarly, we form the inverse change of frame as:

In [19]:
M.set_change_of_frame(es, cart.frame(), 
                      M.change_of_frame(spher.frame(), cart.frame()) * M.change_of_frame(es, spher.frame()),
                      compute_inverse=False)
M.change_of_frame(es, cart.frame())[:, spher]

[cos(ph)*sin(th) sin(ph)*sin(th)         cos(th)]
[cos(ph)*cos(th) cos(th)*sin(ph)        -sin(th)]
[       -sin(ph)         cos(ph)               0]

At this stage, the manifold (user) atlas is

In [20]:
M.atlas()

[Chart (M, (x, y, z)), Chart (M, (r, th, ph))]

The default chart is the first one introduced on the manifold (it can be changed by means of the function `M.set_default_chart`):

In [21]:
M.default_chart()

Chart (M, (x, y, z))

The following vector frames have been introduced on the manifold:

In [22]:
M.frames()

[Coordinate frame (M, (d/dx,d/dy,d/dz)),
 Coordinate frame (M, (d/dr,d/dth,d/dph)),
 Vector frame (M, (e_1,e_2,e_3))]

The default frame is the first one introduced on the manifold (it can be changed by means of the function `M.set_default_frame`):

In [23]:
M.default_frame()

Coordinate frame (M, (d/dx,d/dy,d/dz))

The following changes of frame have been defined:

In [24]:
M.changes_of_frame()

{(Coordinate frame (M, (d/dr,d/dth,d/dph)),
  Coordinate frame (M, (d/dx,d/dy,d/dz))): Field of tangent-space automorphisms on the 3-dimensional differentiable manifold M,
 (Coordinate frame (M, (d/dr,d/dth,d/dph)),
  Vector frame (M, (e_1,e_2,e_3))): Field of tangent-space automorphisms on the 3-dimensional differentiable manifold M,
 (Coordinate frame (M, (d/dx,d/dy,d/dz)),
  Coordinate frame (M, (d/dr,d/dth,d/dph))): Field of tangent-space automorphisms on the 3-dimensional differentiable manifold M,
 (Coordinate frame (M, (d/dx,d/dy,d/dz)),
  Vector frame (M, (e_1,e_2,e_3))): Field of tangent-space automorphisms on the 3-dimensional differentiable manifold M,
 (Vector frame (M, (e_1,e_2,e_3)),
  Coordinate frame (M, (d/dr,d/dth,d/dph))): Field of tangent-space automorphisms on the 3-dimensional differentiable manifold M,
 (Vector frame (M, (e_1,e_2,e_3)),
  Coordinate frame (M, (d/dx,d/dy,d/dz))): Field of tangent-space automorphisms on the 3-dimensional differentiable manifold M}

## Vector field defined in terms of its Cartesian components

We define the vector field $U$ in terms of its components with respect to the default frame, i.e. the Cartesian frame:

In [25]:
U = M.vector_field(name='U')
U[:] = [function('U_x')(x,y,z), function('U_y')(x,y,z), function('U_z')(x,y,z)]
U.display()

U = U_x(x, y, z) d/dx + U_y(x, y, z) d/dy + U_z(x, y, z) d/dz

We can ask for its components in terms of the spherical orthonormal frame:

In [26]:
U.display(es)

U = (x*U_x(x, y, z) + y*U_y(x, y, z) + z*U_z(x, y, z))/sqrt(x^2 + y^2 + z^2) e_1 - (x^2*U_z(x, y, z) + y^2*U_z(x, y, z) - (x*U_x(x, y, z) + y*U_y(x, y, z))*z)/(sqrt(x^2 + y^2 + z^2)*sqrt(x^2 + y^2)) e_2 - (y*U_x(x, y, z) - x*U_y(x, y, z))/sqrt(x^2 + y^2) e_3

In [27]:
U.display_comp(es)

U^1 = (x*U_x(x, y, z) + y*U_y(x, y, z) + z*U_z(x, y, z))/sqrt(x^2 + y^2 + z^2) 
U^2 = -(x^2*U_z(x, y, z) + y^2*U_z(x, y, z) - (x*U_x(x, y, z) + y*U_y(x, y, z))*z)/(sqrt(x^2 + y^2 + z^2)*sqrt(x^2 + y^2)) 
U^3 = -(y*U_x(x, y, z) - x*U_y(x, y, z))/sqrt(x^2 + y^2)

The above components are displayed in terms of the default chart (Cartesian coordinates). If we want them in terms of spherical coordinates, we have to specify it, by setting the second argument of the function `display` to `spher`:

In [28]:
U.display(es, spher)

U = (U_x(r*cos(ph)*sin(th), r*sin(ph)*sin(th), r*cos(th))*cos(ph)*sin(th) + U_y(r*cos(ph)*sin(th), r*sin(ph)*sin(th), r*cos(th))*sin(ph)*sin(th) + U_z(r*cos(ph)*sin(th), r*sin(ph)*sin(th), r*cos(th))*cos(th)) e_1 + (U_x(r*cos(ph)*sin(th), r*sin(ph)*sin(th), r*cos(th))*cos(ph)*cos(th) + U_y(r*cos(ph)*sin(th), r*sin(ph)*sin(th), r*cos(th))*cos(th)*sin(ph) - U_z(r*cos(ph)*sin(th), r*sin(ph)*sin(th), r*cos(th))*sin(th)) e_2 + (U_y(r*cos(ph)*sin(th), r*sin(ph)*sin(th), r*cos(th))*cos(ph) - U_x(r*cos(ph)*sin(th), r*sin(ph)*sin(th), r*cos(th))*sin(ph)) e_3

In [29]:
U.display_comp(es, spher)

U^1 = U_x(r*cos(ph)*sin(th), r*sin(ph)*sin(th), r*cos(th))*cos(ph)*sin(th) + U_y(r*cos(ph)*sin(th), r*sin(ph)*sin(th), r*cos(th))*sin(ph)*sin(th) + U_z(r*cos(ph)*sin(th), r*sin(ph)*sin(th), r*cos(th))*cos(th) 
U^2 = U_x(r*cos(ph)*sin(th), r*sin(ph)*sin(th), r*cos(th))*cos(ph)*cos(th) + U_y(r*cos(ph)*sin(th), r*sin(ph)*sin(th), r*cos(th))*cos(th)*sin(ph) - U_z(r*cos(ph)*sin(th), r*sin(ph)*sin(th), r*cos(th))*sin(th) 
U^3 = U_y(r*cos(ph)*sin(th), r*sin(ph)*sin(th), r*cos(th))*cos(ph) - U_x(r*cos(ph)*sin(th), r*sin(ph)*sin(th), r*cos(th))*sin(ph)

We may also ask for the components of $U$ w.r.t. the natural frame of spherical coordinates:

In [30]:
U.display(spher.frame())

U = (x*U_x(x, y, z) + y*U_y(x, y, z) + z*U_z(x, y, z))/sqrt(x^2 + y^2 + z^2) d/dr - (x^2*U_z(x, y, z) + y^2*U_z(x, y, z) - (x*U_x(x, y, z) + y*U_y(x, y, z))*z)*sqrt(x^2 + y^2)/(x^4 + 2*x^2*y^2 + y^4 + (x^2 + y^2)*z^2) d/dth - (y*U_x(x, y, z) - x*U_y(x, y, z))/(x^2 + y^2) d/dph

In [31]:
U.display(spher.frame(), spher)

U = (U_x(r*cos(ph)*sin(th), r*sin(ph)*sin(th), r*cos(th))*cos(ph)*sin(th) + U_y(r*cos(ph)*sin(th), r*sin(ph)*sin(th), r*cos(th))*sin(ph)*sin(th) + U_z(r*cos(ph)*sin(th), r*sin(ph)*sin(th), r*cos(th))*cos(th)) d/dr + (U_x(r*cos(ph)*sin(th), r*sin(ph)*sin(th), r*cos(th))*cos(ph)*cos(th) + U_y(r*cos(ph)*sin(th), r*sin(ph)*sin(th), r*cos(th))*cos(th)*sin(ph) - U_z(r*cos(ph)*sin(th), r*sin(ph)*sin(th), r*cos(th))*sin(th))/r d/dth + (U_y(r*cos(ph)*sin(th), r*sin(ph)*sin(th), r*cos(th))*cos(ph) - U_x(r*cos(ph)*sin(th), r*sin(ph)*sin(th), r*cos(th))*sin(ph))/(r*sin(th)) d/dph

## Vector field defined in terms of its spherical components

Let us consider a vector field $V$ defined by its components with respect to the orthonormal spherical frame `es` = $(e_1, e_2, e_3)$:

In [32]:
V = M.vector_field(name='V')
V[es,:,spher] = [function('V_1')(r,th,ph), function('V_2')(r,th,ph), function('V_3')(r,th,ph)]
V.display(es, spher)

V = V_1(r, th, ph) e_1 + V_2(r, th, ph) e_2 + V_3(r, th, ph) e_3

We may ask for the components of this vector field with respect to the Cartesian frame (first argument `cart.frame()`), each component being expressed in terms of spherical coordinates (second argument `spher`):

In [33]:
V.display(cart.frame(), spher)

V = (V_2(r, th, ph)*cos(ph)*cos(th) + V_1(r, th, ph)*cos(ph)*sin(th) - V_3(r, th, ph)*sin(ph)) d/dx + (V_2(r, th, ph)*cos(th)*sin(ph) + V_1(r, th, ph)*sin(ph)*sin(th) + V_3(r, th, ph)*cos(ph)) d/dy + (V_1(r, th, ph)*cos(th) - V_2(r, th, ph)*sin(th)) d/dz

In [34]:
V.display_comp(cart.frame(), spher)

V^x = V_2(r, th, ph)*cos(ph)*cos(th) + V_1(r, th, ph)*cos(ph)*sin(th) - V_3(r, th, ph)*sin(ph) 
V^y = V_2(r, th, ph)*cos(th)*sin(ph) + V_1(r, th, ph)*sin(ph)*sin(th) + V_3(r, th, ph)*cos(ph) 
V^z = V_1(r, th, ph)*cos(th) - V_2(r, th, ph)*sin(th)

## Euclidean metric

The standard Euclidean metric is introduced as a Riemannian metric on $M$, whose components with respect to the Cartesian frame are $\mathrm{diag}(1,1,1)$:

In [35]:
g = M.riemannian_metric('g')
g[1,1], g[2,2], g[3,3] = 1, 1, 1
g.display()

g = dx*dx + dy*dy + dz*dz

The components of $g$ with respect to the spherical coordinates are then:

In [36]:
g.display(spher.frame(), spher)

g = dr*dr + r^2 dth*dth + r^2*sin(th)^2 dph*dph

Since `es` = $(e_1,e_2,e_3)$ is an orthonormal frame, the components of $g$ with respect to it are $\mathrm{diag}(1,1,1)$:

In [37]:
g.display(es, spher)

g = e^1*e^1 + e^2*e^2 + e^3*e^3

The covariant derivative operator $\nabla$ is introduced as the (Levi-Civita) connection associated with $g$: 

In [38]:
nabla = g.connection()
print(nabla)
nabla

Levi-Civita connection nabla_g associated with the Riemannian metric g on the 3-dimensional differentiable manifold M


Levi-Civita connection nabla_g associated with the Riemannian metric g on the 3-dimensional differentiable manifold M

The connection coefficient with respect to the natural frame of spherical coordinates (Christoffel symbols) are:

In [39]:
nabla.display(spher.frame(), spher)

Gam^r_th,th = -r 
Gam^r_ph,ph = -r*sin(th)^2 
Gam^th_r,th = 1/r 
Gam^th_th,r = 1/r 
Gam^th_ph,ph = -cos(th)*sin(th) 
Gam^ph_r,ph = 1/r 
Gam^ph_th,ph = cos(th)/sin(th) 
Gam^ph_ph,r = 1/r 
Gam^ph_ph,th = cos(th)/sin(th)

while those with respect to the orthonormal spherical frame are:

In [40]:
nabla.display(es, spher)

Gam^1_22 = -1/r 
Gam^1_33 = -1/r 
Gam^2_12 = 1/r 
Gam^2_33 = -cos(th)/(r*sin(th)) 
Gam^3_13 = 1/r 
Gam^3_23 = cos(th)/(r*sin(th))

The covariant derivative of $U$ is

In [41]:
nabU = nabla(U)
print(nabU)

Tensor field nabla_g(U) of type (1,1) on the 3-dimensional differentiable manifold M


In [42]:
nabU.display()

nabla_g(U) = d(U_x)/dx d/dx*dx + d(U_x)/dy d/dx*dy + d(U_x)/dz d/dx*dz + d(U_y)/dx d/dy*dx + d(U_y)/dy d/dy*dy + d(U_y)/dz d/dy*dz + d(U_z)/dx d/dz*dx + d(U_z)/dy d/dz*dy + d(U_z)/dz d/dz*dz

while the covariant derivative of $V$ is

In [43]:
nabV = nabla(V)
print(nabV)

Tensor field nabla_g(V) of type (1,1) on the 3-dimensional differentiable manifold M


In [44]:
nabV.display(es, spher)

nabla_g(V) = d(V_1)/dr e_1*e^1 - (V_2(r, th, ph) - d(V_1)/dth)/r e_1*e^2 - (V_3(r, th, ph)*sin(th) - d(V_1)/dph)/(r*sin(th)) e_1*e^3 + d(V_2)/dr e_2*e^1 + (V_1(r, th, ph) + d(V_2)/dth)/r e_2*e^2 - (V_3(r, th, ph)*cos(th) - d(V_2)/dph)/(r*sin(th)) e_2*e^3 + d(V_3)/dr e_3*e^1 + d(V_3)/dth/r e_3*e^2 + (V_2(r, th, ph)*cos(th) + V_1(r, th, ph)*sin(th) + d(V_3)/dph)/(r*sin(th)) e_3*e^3